## Installing and Importing Dependencies

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
import nltk
import re
import pandas as pd
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Furqan970\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Defining the functions

In [2]:
# Define a function to remove repeated characters
def remove_repeated_chars(text):
    # Use regular expressions to remove repeated ! and ...
    text = re.sub(r'(!)\1+', r'\1', text)
    text = re.sub(r'(\.)\1+', r'\1', text)
    return text

## Loading Pre-trained BERT Model and Tokenizer

In [3]:
# Define the pre-trained BERT model and tokenizer
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

## Defining Reviews and Thresholds

In [4]:
reviews = [
    "I absolutely loved the mouse and it's great for gaming, however, I developed a scroll wheel issue and after countless tries of fixing it I have still gotten nowhere, the issue is that while scrolling the scroll wheel constantly skips in the other direction and makes itself pretty unusable for browsing website or scrolling through items in a game, other than that it's a genuinely great product but with this issue being as prevalent as it is, I can't recommend it for the price.",
    "I have been using Logitech mouse for almost 5 years now, I think Logitech seriously needs to check their switch quality, I mean we are spending 13k - 15k for a mouse and just within a few months almost all Logitech mouse gets this double-clicking issue, I have replaced my Logitech mouse like 6 times, even my G502 wireless had the same issues. I was a fool to buy another G502 wireless, and guess what same issue persists. Bought this mouse thinking pro gamer use it so there might not be any quality issues. Seems like these are just for advertising they actually might not be using it. Even the Logitech G pro keyboard keys are having this multiple keypressing issue. I think this would be my last purchase of Logitech mouse. I am done!!! I WOULD RECOMMEND NOT TO BUY ANY LOGITECH PRODUCT PLEASE DON'T WASTE YOUR MONEY!!!!!",
    "So initially bought this mouse around the end of November 2021. Great mouse, very light for FPS gaming, fairly comfortable although it might be slippery if you have sweaty palms. The stock glides are decent but could use an upgrade to better ones. Battery life is alright I found myself charging it every 2-3 weeks or so, and I am a pretty heavy user. Micro USB for charging in 2021-2022 is a crime against humanity, just give us type-c Logitech. Now the disappointment comes, as I started having tracking issues, constant disconnect/ lag with my PC even though my PC is right next to my mouse. All the firmware for the mouse, drivers, windows updated, hence the problem is still there. I guess I was just unlucky this mouse started developing issues.",
    "I play a fair bit of games, I prefer shooters the most. I replaced a Zowie XL & I'm 6'3 so have quite large hands. The packaging was very minimal but slick. Straight out of the box I instantly took notice of how light the mouse felt, it feels like nothing. The device had 48% charge out of the box. There is no fiddling around with connecting the device, it's just plug and play. The size of the mouse is perfect, still have palm grip & it fits very very nicely for me. You will require the software Logitech G Hub to modify the mouse DPI settings if you want to do, or set up profiles. I already had this software installed for my Logitech webcam, so anyone already using Logitech devices most likely already uses the software. When it comes to playing games, the main FPS I play I felt like the sensitivity felt a bit higher even on the same settings as my previous mouse. I'm still modifying those settings to see what feels best, but I would honestly say the performance does feel a lot smoother as well. I wasn't expecting too much of an increase in how smooth it would feel, I had replaced my previous mouse due to its scroller not working. I haven't tried the glide pad that can be changed over for more glide or any of the grips. The scroller wheel feels awesome too. So far this is an absolute 10/10 mouse & I would highly recommend it.",
    "Coming from an old Logitech G502 that served me well for many years, across many games, I wasn't sure the 'light' trend of mice was for me. I'm of course now a believer, and I love the glide feel and how easy it is to use this mouse every day. Gaming and for work. I weighed up a lot of expensive options in this category, but I'm happy with the G Pro X. Comparing to the G502, this has fewer buttons and feels less sturdy. You are in fact getting less of a product. But that's the point, as FPS games require quick movements, you need less weight to flick around. I'm just not sure the value is worth it for the price... but that's not to say I'm unhappy with the product. The optical sensor is really good, and I just feel more precise in my movement. Buttons have a good click to them. Scroll wheel feels a little cheap though. It kinda feels like something from a $20 mouse, not to mention there isn't a button to allow free scrolling, which I used often on my G502. I'm used to this new mouse now, just that I'd be more impressed if they sold this cheaper. Otherwise for a gaming setup, it definitely deserves to be top tier.",
    "Brilliant and lightweight and very easy to setup and use. Just go for it if you have the budget.",
    "It stopped working within 6 months (scroll & click issue) and the replacement mouse also stopped within 4 months! (same scroll & click issue ). Pathetic product by Logitech at this price point."
]

In [5]:
# Define threshold values for sentiment classification
positive_threshold = 0.5  # Adjust as needed
negative_threshold = 0.5  # Adjust as needed

## Performing Sentiment Analysis

In [6]:
# Create empty lists to store review text, overall sentiments, and overall sentiment scores
review_text = []
overall_sentiments = []
overall_sentiment_scores = []

# Perform sentiment analysis on each sentence in each review and calculate an overall sentiment for each review
for i, review in enumerate(reviews):
    # Split the review into sentences
    sentences = sent_tokenize(review)
    
    print(f"Sentiment analysis for Review {i + 1}:")
    overall_sentiment_score = 0.0
    sentiment_counts = {"Negative": 0, "Neutral": 0, "Positive": 0}
    
    for j, sentence in enumerate(sentences):
        # Remove repeated characters
        sentence = remove_repeated_chars(sentence)
        
        tokens = tokenizer(sentence, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**tokens)
        logits = outputs.logits
        probabilities = softmax(logits, dim=1)
        sentiment_label = torch.argmax(probabilities, dim=1).item()

        # Interpret sentiment label (0: negative, 1: neutral, 2: positive)
        sentiment = "Negative" if sentiment_label == 0 else "Neutral" if sentiment_label == 1 else "Positive"
        sentiment_score = probabilities[0][sentiment_label].item()
        
        print(f"Sentence {j + 1}:")
        print(sentence)
        print(f"Sentiment: {sentiment}")
        print(f"Sentiment Score: {sentiment_score:.4f}\n")
        
        # Update sentiment counts
        sentiment_counts[sentiment] += 1
        
        # Add the sentiment score to the overall sentiment score
        overall_sentiment_score += sentiment_score
    
    # Calculate and display the overall sentiment for the review based on counts
    overall_sentiment = max(sentiment_counts, key=sentiment_counts.get)
    overall_sentiment_score /= len(sentences)  # Calculate the average sentiment score
    print(f"Overall Review Sentiment: {overall_sentiment}")
    print(f"Sentiment Counts for Review {i + 1}: {sentiment_counts}")
    print(f"Overall Sentiment Score for Review {i + 1}: {overall_sentiment_score:.4f}\n")
    
    # Append data to the lists
    review_text.append(review)
    overall_sentiments.append(overall_sentiment)
    overall_sentiment_scores.append(overall_sentiment_score)  # Append the overall sentiment score

Sentiment analysis for Review 1:
Sentence 1:
I absolutely loved the mouse and it's great for gaming, however, I developed a scroll wheel issue and after countless tries of fixing it I have still gotten nowhere, the issue is that while scrolling the scroll wheel constantly skips in the other direction and makes itself pretty unusable for browsing website or scrolling through items in a game, other than that it's a genuinely great product but with this issue being as prevalent as it is, I can't recommend it for the price.
Sentiment: Neutral
Sentiment Score: 0.5406

Overall Review Sentiment: Neutral
Sentiment Counts for Review 1: {'Negative': 0, 'Neutral': 1, 'Positive': 0}
Overall Sentiment Score for Review 1: 0.5406

Sentiment analysis for Review 2:
Sentence 1:
I have been using Logitech mouse for almost 5 years now, I think Logitech seriously needs to check their switch quality, I mean we are spending 13k - 15k for a mouse and just within a few months almost all Logitech mouse gets thi

## Showing as a DataFrame

In [7]:
# Create a DataFrame to display the results
df = pd.DataFrame({
    'Review': review_text,
    'Overall Sentiment': overall_sentiments,
    'Overall Sentiment Score': overall_sentiment_scores  # Add the overall sentiment score to the DataFrame
})

# Display the DataFrame
df

Review Overall Sentiment  \
0  I absolutely loved the mouse and it's great fo...           Neutral   
1  I have been using Logitech mouse for almost 5 ...          Positive   
2  So initially bought this mouse around the end ...           Neutral   
3  I play a fair bit of games, I prefer shooters ...          Positive   
4  Coming from an old Logitech G502 that served m...          Positive   
5  Brilliant and lightweight and very easy to set...          Positive   
6  It stopped working within 6 months (scroll & c...          Negative   

   Overall Sentiment Score  
0                 0.540559  
1                 0.496537  
2                 0.488516  
3                 0.518103  
4                 0.513582  
5                 0.681462  
6                 0.601361